In [2]:
import warnings
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense

In [3]:
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True, linewidth=150, precision=2)

In [4]:
print("GPU Available: ", tf.test.is_gpu_available())
print(tf.__version__)

GPU Available:  True
2.0.0


In [ ]:
# warnings.filterwarnings('ignore')
tf.keras.backend.clear_session()  # For easy reset of notebook state.
np.set_printoptions(suppress=True, linewidth=120, precision=2)

In [ ]:
def perf_measure(y_true, y_pred):
    
    cnf_matrix = confusion_matrix(y_true, y_pred)
    
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)
    
    FSCORE = np.divide((2*PPV*TPR), (PPV+TPR))
    
    return PPV, TPR, FSCORE, FNR, FPR, TNR

In [ ]:
name_of_particle = 'JetHTs'

X_train = np.load("matrices/" + name_of_particle +"_train.npy",)
y_train = np.load("matrices/" + name_of_particle +"_y_train.npy",)
X_val = np.load("matrices/" + name_of_particle +"_val.npy",)
y_val = np.load("matrices/" + name_of_particle +"_y_val.npy",)
X_test = np.load("matrices/" + name_of_particle +"_test.npy",)
y_test = np.load("matrices/" + name_of_particle +"_y_test.npy",)
X_train = X_train[:, :-3]
X_val = X_val[:, :-3]
X_test = X_test[:, :-3]
_, V = X_train.shape
K = 1
V

In [ ]:
X_train_pos = X_train[np.where(y_train==0)]
X_train_neg = X_train[np.where(y_train==1)]
y_train_pos = y_train[np.where(y_train==0)]
y_train_neg = y_train[np.where(y_train==1)]

# X_val_pos = X_val[np.where(y_val==0)]
# X_val_neg = X_val[np.where(y_val==1)]
# y_val_pos = y_val[np.where(y_val==0)]
# y_val_neg = y_val[np.where(y_val==1)]

# X_test_pos = X_test[np.where(y_test==0)]
# X_test_neg = X_test[np.where(y_test==1)]
# y_test_pos = y_test[np.where(y_test==0)]
# y_test_neg = y_test[np.where(y_test==1)]

X_train_pos.shape[0] + X_train_neg.shape[0] == X_train.shape[0]

In [ ]:
X_train_pos = tf.convert_to_tensor(X_train_pos)
y_train_pos = tf.convert_to_tensor(y_train_pos)
X_train_neg = tf.convert_to_tensor(X_train_neg)
y_train_neg = tf.convert_to_tensor(y_train_neg)

X_val = tf.convert_to_tensor(X_val)
y_val = tf.convert_to_tensor(y_val)

X_test = tf.convert_to_tensor(X_test)
y_test = tf.convert_to_tensor(y_test)

# X_val_pos = tf.convert_to_tensor(X_val_pos)
# y_val_pos = tf.convert_to_tensor(y_val_pos)
# X_val_neg = tf.convert_to_tensor(X_val_neg)
# y_val_neg = tf.convert_to_tensor(y_val_neg)


# X_test_pos = tf.convert_to_tensor(X_test_pos[1:10, :])
# y_test_pos = tf.convert_to_tensor(y_test_pos[1:10])
# X_test_neg = tf.convert_to_tensor(X_test_neg[1:10, :])
# y_test_neg = tf.convert_to_tensor(y_test_neg[1:10])

In [ ]:
# batch_size = 512

# test_ds_pos = tf.data.Dataset.from_tensor_slices((X_test_pos, y_test_pos)).batch(batch_size) #.shuffle(1000)
# test_ds_neg = tf.data.Dataset.from_tensor_slices((X_test_neg, y_test_neg)).batch(batch_size) #.shuffle(1000)
# test_ds = (test_ds_pos, test_ds_neg)
    
# train_ds_pos = tf.data.Dataset.from_tensor_slices((X_train_pos, y_train_pos)).batch(batch_size) #.shuffle(1000)
# train_ds_neg = tf.data.Dataset.from_tensor_slices((X_train_neg, y_train_neg)).batch(batch_size) #.shuffle(1000)
# train_ds = (train_ds_pos, train_ds_neg)

# val_ds_pos = tf.data.Dataset.from_tensor_slices((X_val_pos, y_val_pos)).batch(batch_size) #.shuffle(1000)
# val_ds_neg = tf.data.Dataset.from_tensor_slices((X_val_neg, y_val_neg)).batch(batch_size) #.shuffle(1000)
# val_ds = (val_ds_pos, val_ds_neg)


### Neural Network

### Brute-forth

#### Creating the custom loss function for training: 

In [ ]:
def loss_bf(x_pos, x_neg):    
    
    p_pos = nn(x_pos)
    p_neg = nn(x_neg)
    
    N_pos, V = x_pos.shape
    N_neg, _ = x_neg.shape
    N_pseudo = N_pos
    
    supp_min = tf.reduce_min(x_pos, keepdims=True, axis=0)
    supp_max = tf.reduce_max(x_pos, keepdims=True, axis=0)
    
    x_pseudo = tf.random_uniform(shape=[N_pseudo, V],
                                 minval=supp_min-1, maxval=supp_max+1,
                                 dtype='float64',)
    
    p_psedo = model_occ(x_pseudo)
    print("p_psedo", p_psedo)
    
    N_pos = x_pos.shape[0]
    N_neg = x_neg.shape[0]
    
    loss_pos = N_pos / (N_pos + N_neg) * tf.reduce_mean(tf.nn.softplus(-p_pos))
    loss_neg = N_neg / (N_pos + N_neg) * tf.reduce_mean(tf.nn.softplus(p_neg))
    loss_pseudo = 0.01 * tf.reduce_mean(tf.nn.softplus(p_psedo))
    
    print("loss_bf:", loss_pos + loss_neg)
    
    return loss_pos + loss_neg

In [ ]:
loss_bf(X_test, X_test)

#### Chosing an optimizer 

In [ ]:
# loss_object = tf.keras.losses.BinaryCrossentropy()  (*)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6) 

#### metrics to measure the loss and accuracy of the model

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss') 
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')

In [ ]:
@tf.function
def train_step(x_pos, x_neg, labels):
    with tf.GradientTape() as tape:  # Record operations for automatic differentiation
        predictions = compute_gradients()
        
    gradients = tape.gradient(predicions, model_occ.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model_occ.trainable_variables))
#     train_loss(loss)

In [ ]:
@tf.function
def test_step(X, labels):
    predictions = model_occ(X)
    labels = tf.reshape(tf.tile(labels, [2]), [-1, 2])
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 50

training_losses_occ, training_accuracies_occ = [], []
validations_losses_occ, validations_accuracies_occ = [], []

for epoch in range(EPOCHS):
    results = train_step(x_pos=X_train_pos, x_neg=X_train_neg, labels=y_train)
    test_step(X_val, y_val)
    
#     validations_losses_occ.append(test_loss.result().numpy())
#     validations_accuracies_occ.append(test_accuracy.result().numpy())
    
#     training_losses_occ.append(train_loss.result().numpy())
#     training_accuracies_occ.append(train_accuracy.result().numpy())

#     validations_losses_occ.append(test_loss.result().numpy())
#     validations_accuracies_occ.append(test_accuracy.result().numpy())
    
    template = 'Epoch {}, Validation Loss: {:.3f}, Validation Accuracy:{:.3f},'
    
    print (template.format(epoch+1,
                           test_loss.result().numpy(),
                           test_accuracy.result().numpy()*100),)
    
model_occ.save_weights("NN-ckecks/OCC_Bope"+ name_of_particle +".h5")

In [ ]:
model_occ.summary()

In [ ]:
model_occ.summary()

In [ ]:
new_model_occ = get_model()
new_model_occ.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6))

# Since In this implementation instead of weight we are dealing 
# with codes and classes therefore the traditional serialization and
# deserialization is not possible. So we have to first initialze
# the model (which is code) and then load the weights 
# Ref: https://colab.research.google.com/drive/172D4jishSgE3N7AO6U2OKAA_0wNnrMOq#scrollTo=OOSGiSkHTERy

cntr = 0
for i, j in train_ds:
    if cntr == 0:
        new_model_occ.train_on_batch(i[:1], j[:1])
    cntr += 1 

# new_model_occ.load_weights('NN-ckecks/ThreeLayerNN_model'+ name_of_particle+'.h5')
test_predictions = new_model_occ.predict(X_test)
probabilities = tf.nn.sigmoid(test_predictions)
labels_pred_occ = tf.argmax(probabilities, axis=1)


labels_true_occ = []
for i, j in test_ds:
    for k in j.numpy():
        labels_true_occ.append(k)

f1_score_occ = precision_recall_fscore_support(labels_true_occ, labels_pred_occ, average='weighted') # Does not take into account labels imbalanced
print("precision:", "%.2f" %f1_score_occ[0], "recall:", "%.2f" % f1_score_occ[1], "fscore:", "%.2f" %f1_score_occ[2])


In [ ]:
y_true = tf.constant([[1, 0, 1, 1, 0]])
y_true.numpy()

In [ ]:
np.mean((1*y_true).numpy(), axis=-1)

In [ ]:
X = np.array([[1, 2, 3], [4, 5, 6]])
Y = np.array([0, 1])
n_features = 3

n_neurons_in_h1 = 2


In [ ]:
x = tf.convert_to_tensor(X, dtype=tf.float32, name='X')
y = tf.convert_to_tensor(Y, dtype=tf.float32, name='y')

W1 = tf.random.normal(shape=[n_features, n_neurons_in_h1], 
                      mean=0.0, stddev=1/np.sqrt(n_features), name='W1')
b1 = tf.random.normal(shape=[n_neurons_in_h1],
                      mean=0.0, stddev=1/np.sqrt(n_features), name='b1')

In [ ]:
x

In [ ]:
W1

In [ ]:
b1

In [ ]:
W1_ = tf.Variable([[1, -1], [2, -2], [3, -3]], dtype='float32',name='W1')

b1_ = tf.Variable([0.4, 0.9], name='b1')

In [ ]:
print(W1_)
print(b1_)

In [ ]:
tf.matmul(x, W1_)+ b1_

In [ ]:
tf.sigmoid(tf.matmul(x, W1_) + b1_)

In [ ]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x
    dy_dx = g.gradient(y, x)
print(g.watch(x))
print(dy_dx)
